In [7]:
from datetime import datetime
import requests
import json
from gql import gql, Client
import pandas as pd
import time
import numpy as np
from sys import argv

from gql.transport.requests import RequestsHTTPTransport

In [41]:
mystring = """ {{
  orders(
    first: 1000
    orderBy: updatedAt
    orderDirection: asc
    subgraphError: allow
    where: {{status: open, updatedAt_gt:"{0}", expiresAt_gt: "{1}", category: parcel}}
  ) {{
    id
    price
    updatedAt
    createdAt
    expiresAt
    owner
    nft {{
      parcel {{
        id
        x
        y
      }}
    }}
  }}
}}
"""

In [8]:
# Select your transport with a defined url endpoint
transport = RequestsHTTPTransport(url="https://api.thegraph.com/subgraphs/name/decentraland/marketplace")
# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

In [51]:
df

,id,price,updatedAt,createdAt,expiresAt,owner,nft,parcel,updatedAt_dt,x,y,parcel_id
0,0x7342d2fbf05b520afd2985fac3fb5457624033d7a513...,27979.0,1540234199,1540234199,1.669075e+09,0x5b4fd404c33190d8d58a42e8714c773def75d7f0,{'parcel': {'id': 'parcel-0xf87e31492faf9a91b0...,{'id': 'parcel-0xf87e31492faf9a91b02ee0deaad50...,2018-10-22,-23,-21,parcel-0xf87e31492faf9a91b02ee0deaad50d51d56d5...
1,0xf7219bc9d0f69575816e790e6c554e34fe9ba0978d8e...,10000000.0,1555182518,1555182518,1.653084e+09,0x0bb75bef057da63a0ae4b25fe9adafd35cd92b87,{'parcel': {'id': 'parcel-0xf87e31492faf9a91b0...,{'id': 'parcel-0xf87e31492faf9a91b02ee0deaad50...,2019-04-13,-32,72,parcel-0xf87e31492faf9a91b02ee0deaad50d51d56d5...
2,0x11dc7680a31fec2227388a0728592e28ef24aaa4c8b8...,10000000.0,1558294561,1558294561,1.704089e+09,0x4a5c04e65a86b49c9783a1e6ad795d0e8fdaeb6b,{'parcel': {'id': 'parcel-0xf87e31492faf9a91b0...,{'id': 'parcel-0xf87e31492faf9a91b02ee0deaad50...,2019-05-19,-55,-16,parcel-0xf87e31492faf9a91b02ee0deaad50d51d56d5...
3,0x339f6a367908324629e85ca421466d8a66a433f984e5...,90210.0,1570853465,1570853465,1.704042e+09,0x288010917743ab792c491e6c6653f8c2445174bf,{'parcel': {'id': 'parcel-0xf87e31492faf9a91b0...,{'id': 'parcel-0xf87e31492faf9a91b02ee0deaad50...,2019-10-12,50,-111,parcel-0xf87e31492faf9a91b02ee0deaad50d51d56d5...
4,0xa4768e8b7647947a3f8dcffb744916642516af014735...,50000000.0,1580932887,1580932887,1.678205e+09,0x32c909157c5889ed6890da21055e4cea8ea91a1c,{'parcel': {'id': 'parcel-0xf87e31492faf9a91b0...,{'id': 'parcel-0xf87e31492faf9a91b02ee0deaad50...,2020-02-05,4,-32,parcel-0xf87e31492faf9a91b02ee0deaad50d51d56d5...
...,...,...,...,...,...,...,...,...,...,...,...,...
623,0x8cc0ceb9ead8f1c6c59881c4c72fb41496057003df61...,5000.0,1651154037,1651154037,1.653696e+09,0xe010b3bcbb3d077a82985194800227914393360a,{'parcel': {'id': 'parcel-0xf87e31492faf9a91b0...,{'id': 'parcel-0xf87e31492faf9a91b02ee0deaad50...,2022-04-28,149,-147,parcel-0xf87e31492faf9a91b02ee0deaad50d51d56d5...
624,0xf4bf6a61522b2ba7ae51e53cdde543889dfce4033cb4...,3999.0,1651154123,1651154123,1.653696e+09,0xa01424b7540adbb792375dcf97b733a5d68ad347,{'parcel': {'id': 'parcel-0xf87e31492faf9a91b0...,{'id': 'parcel-0xf87e31492faf9a91b02ee0deaad50...,2022-04-28,56,-32,parcel-0xf87e31492faf9a91b02ee0deaad50d51d56d5...
625,0xc19983ae99381e20960be9d13184f20ca4256e7e3270...,4400.0,1651161845,1651161845,1.653696e+09,0x5ce213893956bbf4249a7f8a079331280065eec6,{'parcel': {'id': 'parcel-0xf87e31492faf9a91b0...,{'id': 'parcel-0xf87e31492faf9a91b02ee0deaad50...,2022-04-28,-44,-28,parcel-0xf87e31492faf9a91b02ee0deaad50d51d56d5...
626,0xa6050303f59c7d009884b41df89710414ce4889c62a3...,4999.0,1651177419,1651177419,1.653696e+09,0x35152346a603ec69b96c6e3d8478b8d80d51aafa,{'parcel': {'id': 'parcel-0xf87e31492faf9a91b0...,{'id': 'parcel-0xf87e31492faf9a91b02ee0deaad50...,2022-04-28,-46,83,parcel-0xf87e31492faf9a91b02ee0deaad50d51d56d5...


In [42]:
df = pd.DataFrame()

#update parameter used in mystring to start querying the database at the earliest update date of sale. The update 
#date is specified in epoch date and needs to be converted to datetime for human consumption.
update = 1
now = datetime.now().strftime('%s') + '000'

while True:
    
    #query the data using GraphQL python library.
    query = gql(mystring.format(update, now))
    result = client.execute(query)
    
    #if there is no data returned it means you reached the end and should stop querying.
    if len(client.execute(query)['orders']) <= 1:
        break
   
    else:
        #Create a temporary dataframe to later append to the final dataframe that compiles all 1000-row dataframes.
        df1 = pd.DataFrame()
        df1 = pd.DataFrame(result['orders'])
        #unfold a subdict into a series of columns.
        df1 = df1.join(df1['nft'].apply(pd.Series),lsuffix='_1',rsuffix='_2')     
        
        #append your temp dataframe to your master dataset.
        df = df.append(df1)
        
        #Pass into the API the max date from your dataset to fetch the next 1000 records.
        update = df['updatedAt'].max()
        print("last updated at: {}".format(time.strftime('%Y-%m-%d', time.localtime(int(update)))))

#reformat the update date in human-readable datetime format.
df['price'] = df['price'].astype(float)/1e18
df['expiresAt'] = df['expiresAt'].astype(float)/1e3
df['updatedAt_dt'] = df['updatedAt'].apply(lambda x: time.strftime('%Y-%m-%d', time.localtime(int(x))) )
df['x'] = df.parcel.apply(lambda a: a['x'])
df['y'] = df.parcel.apply(lambda a: a['y'])
df['parcel_id'] = df.parcel.apply(lambda a: a['id'])
df.to_csv('on_sale_parcels.csv',index=False)

last updated at: 2022-04-28


In [ ]:
## Query string used to get latest parcel transaction information
mystring = """
{{
  orders (first: 1000 orderBy: updatedAt, orderDirection: asc where: {{ status:sold category:parcel updatedAt_gt:"{0}"}}, subgraphError: allow) {{
 
 
    category
    status
    price
    id
    updatedAt
    nft {{
      owner {{
        id
      }}
      name
      estate {{
        id
      }}
      parcel {{
        id
        x
        y
      }}
     
      owner {{
        id
      }}
    }}
   
  }}
 
}}"""

In [ ]:
df = pd.DataFrame()

#update parameter used in mystring to start querying the database at the earliest update date of sale. The update 
#date is specified in epoch date and needs to be converted to datetime for human consumption.
update = 1

while True:
    
    #query the data using GraphQL python library.
    query = gql(mystring.format(update))
    result = client.execute(query)
    
    #if there is no data returned it means you reached the end and should stop querying.
    if len(client.execute(query)['orders']) <= 1:
        break
   
    else:
        #Create a temporary dataframe to later append to the final dataframe that compiles all 1000-row dataframes.
        df1 = pd.DataFrame()
        df1 = pd.DataFrame(result['orders'])
        #unfold a subdict into a series of columns.
        df1 = df1.join(df1['nft'].apply(pd.Series),lsuffix='_1',rsuffix='_2')     
        
        #append your temp dataframe to your master dataset.
        df = df.append(df1)
        
        #Pass into the API the max date from your dataset to fetch the next 1000 records.
        update = df['updatedAt'].max()
        print("last updated at: {}".format(time.strftime('%Y-%m-%d', time.localtime(int(update)))))

#reformat the update date in human-readable datetime format.
df['updatedAt_dt'] = df['updatedAt'].apply(lambda x: time.strftime('%Y-%m-%d', time.localtime(int(x))) )
df['price'] = df['price'].astype(float)/1e18
df['owner'] = df.owner.apply(lambda x: x['id'])
df['x'] = df.parcel.apply(lambda a: a['x'])
df['y'] = df.parcel.apply(lambda a: a['y'])
df['parcel_id'] = df.parcel.apply(lambda a: a['id'])
df.to_csv('parcel_transactions.csv')

In [ ]:
# String used to query parcel info
mystring = """
{{
  parcels (first: 301 orderBy: x where: {{ y_lte:"{0}" y_gte:"{0}"}} , subgraphError: allow) {{
    id
    tokenId
    owner{{
     id
    }}
    x
    y
    estate{{
     id
    }}
  }}
 
}}"""

In [ ]:
df = pd.DataFrame()

#update parameter used in mystring to start querying the database at the earliest update date of sale. The update 
#date is specified in epoch date and needs to be converted to datetime for human consumption.
counter = 1

for row_num in range(-150,151):
    
    #query the data using GraphQL python library.
    query = gql(mystring.format(row_num))
    result = client.execute(query)
    
    #if there is no data returned it means you reached the end and should stop querying.
    if len(client.execute(query)['parcels']) < 1:
        break
   
    else:
        #Create a temporary dataframe to later append to the final dataframe that compiles all 1000-row dataframes.
        df1 = pd.DataFrame()
        df1 = pd.DataFrame(result['parcels'])
        
        """
        #unfold a subdict into a series of columns.
        df1 = df1.join(df1['nft'].apply(pd.Series),lsuffix='_1',rsuffix='_2')     
        """
        #append your temp dataframe to your master dataset.
        df = df.append(df1)
        
        #Pass into the API the max date from your dataset to fetch the next 1000 records.
        print('first {} rows done!'.format(row_num + 151))
df['owner'] = df.owner.apply(lambda x: x['id'])
df['x'] = df.x.apply(lambda a: int(a))
df['y'] = df.y.apply(lambda a: int(a))
df.to_csv('decentraland_ownership.csv')

In [ ]:
# String used to query estate info
mystring = """{{
  orders (first: 1000 orderBy: updatedAt, orderDirection: asc where: {{ status:sold category:estate updatedAt_gt:"{0}"}}, subgraphError: allow) {{
    category
    price
    id
    buyer
    updatedAt
    nft{{
      estate{{
        id
        parcels{{
        id
        x
        y
        }}
      }}
    }}
  }}
}}"""

In [ ]:
df = pd.DataFrame()

#update parameter used in mystring to start querying the database at the earliest update date of sale. The update 
#date is specified in epoch date and needs to be converted to datetime for human consumption.
update = 1

while True:
    
    #query the data using GraphQL python library.
    query = gql(mystring.format(update))
    result = client.execute(query)
    
    #if there is no data returned it means you reached the end and should stop querying.
    if len(client.execute(query)['orders']) <= 1:
        break
   
    else:
        #Create a temporary dataframe to later append to the final dataframe that compiles all 1000-row dataframes.
        df1 = pd.DataFrame()
        df1 = pd.DataFrame(result['orders'])
        #unfold a subdict into a series of columns.
        #df1 = df1.join(df1['nft'].apply(pd.Series),lsuffix='_1',rsuffix='_2')     
        
        #append your temp dataframe to your master dataset.
        df = df.append(df1)
        
        #Pass into the API the max date from your dataset to fetch the next 1000 records.
        update = df['updatedAt'].max()
        print("last updated at: {}".format(time.strftime('%Y-%m-%d', time.localtime(int(update)))))

#reformat the update date in human-readable datetime format.
df['updatedAt_dt'] = df['updatedAt'].apply(lambda x: time.strftime('%Y-%m-%d', time.localtime(int(x))) )
df['price'] = df['price'].astype(float)/1e18
df['parcels'] = df.nft.apply(lambda x: x['estate']).apply(lambda x: x['parcels']).apply(lambda x: [(int(i['x']),int(i['y'])) for i in x])
df['estate_id'] = df.nft.apply(lambda x: x['estate']).apply(lambda x: x['id'])
df.to_csv('estate_transactions.csv',index=False)